# Web Scraping Used Cars on sgcarmart.com
## 1. Introduction

### This document outlines the process of web scraping data from sgcarmart.com, the largest online car marketplace in Singapore, to analyze the used car market.

### Respecting sgcarmart.com's Rules
The scraping script will adhere to the guidelines outlined in sgcarmart.com's robots.txt file. Here's a summary of the restrictions:

Crawlers must wait at least 5 seconds between requests (Crawl-delay: 5).
Specific directories are off-limits for scraping, including:
cgi-bin/
images/
mail/
dealer/
directory/premium/
includes/
phpads/
update/
upload/

### Data Extraction
The script will focus on extracting the following information for each used car listing:

Car Listing URL 'LISTING_URL', 
Car Brand and Model 'BRAND', 
Price 'PRICE', 
Depreciation Value Yearly 'DEPRE_VALUE_PER_YEAR', 
Registered Date 'REG_DATE', 
Mileage in KM 'MILEAGE_KM', 
Year of Manufacture 'MANUFACTURED_YEAR', 
Road Tax Yearly 'ROAD_TAX_PER_YEAR', 
Automatic or Manual Tranmission 'TRANSMISSION', 
Deregistration Value as of Web Scraping DTD 'DEREG_VALUE_FROM_SCRAPE_DATE', 
Web Scraping DTD 'SCRAPE_DATE', 
Open Market Value (OMV) 'OMV', 
Additional Registration Fee (ARF) 'ARF', 
Certificate of Entitlement (COE) from Web Scraping DTD 'COE_FROM_SCRAPE_DATE', 
Number of Days till COE Expires 'DAYS_OF_COE_LEFT', 
Engine Capacity in CC 'ENGINE_CAPACITY_CC', 
Car Curb Weight in KG 'CURB_WEIGHT_KG', 
Number of Past Owners 'NO_OF_OWNERS', 
Vehicle Type 'VEHICLE_TYPE'

This data will be used for further analysis of the used car market in Singapore.

## 2. Import Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import re
from datetime import datetime

## 3. Pre-defined Functions

In [2]:
# All retriever functions take a parsed individual car lsting url and returns a desired attribute named after the function


# Brand Retriever Function 
 
def brand_retrieval(parsed_url):
    # Find the <a> tag with the "nounderline globaltitle" class
    brand_tag = parsed_url.find("a", class_="nounderline globaltitle")
    
    if brand_tag:
        # Extract the brand name from the href attribute
        brand_name = brand_tag["href"].split("=")[1].replace("+", " ").split("&")[0]
        return brand_name
    else:
        brand_name = np.nan  # Stores NA values as nan
    
    return brand_name
    
# Price Retriever Function

def price_error_handling(data_value):
    # Try-Exception error handling
    
    try:   # First try to deal with values higher than 1000
        price = data_value[1]  # will fail on IndexError if retrieves ['na'] scenario
        price = int(price.split(',')[0] + price.split(',')[1]) # Will fail on IndexError if tries to split '900' with a ',' in ['',900]
        
    except IndexError:  # Dealing with ['na'] and ['', 900'] scenarios
        try: 
            price = int(data_value[1]) # Will fail on IndexError if ['na'] scenario
        except IndexError:  # Deals with ['na'] scenarios
            price = np.nan  # Stores NA values as nan
    
    return price

def price_retrieval(parsed_listing_url):
    
    data_value = parsed_listing_url.find_all(class_="font_red")[0].text.strip()
    data_value = data_value.split('$')
    price = price_error_handling(data_value)
    return price

# Deprecration Value Per Year Retriever Function
def depreciation_value_per_year_error_handler(data_value):
    if len(data_value) < 2:
        data_value = np.nan

    else: 
        data_value = data_value[1].split('/yr')
        try:                 
            desired_value = int(data_value[0].split(',')[0] +\
                                data_value[0].split(',')[1]) # Will fail on IndexError if tries to split '900' with a ',' in ['900','']
        except IndexError: 
            desired_value = int(data_value[0])
        
        return desired_value
    
def depreciation_value_per_year_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_="label")[1].findNextSibling().text.strip().split('$')
    depreciation_value_per_year = depreciation_value_per_year_error_handler(data_value)
    return depreciation_value_per_year

# Road Tax Per Year Retriever
def road_tax_error_handler(string_data):
    if '/yr' in string_data: # Only takes in scenarios that are not NA
        try:
            # Removes '$" character and splits string_data into a list of ['', 1,000] or ['', 900]
            road_tax_per_year = \
            string_data.replace('/yr','').strip().split('$') 

            # Accesses the second item in the list
            road_tax_per_year = road_tax_per_year[1] 


            road_tax_per_year = int(road_tax_per_year.split(',')[0] +\
                                    road_tax_per_year.split(',')[1])  # Will fail on IndexError if value is above 1000

        except IndexError: # Handles values that are below 1000. (i.e. ['',900])
            road_tax_pear_year = int(road_tax_per_year[1])

    else: # Deals with 'NA' scenario
        road_tax_per_year = np.nan
    
    return road_tax_per_year

def road_tax_retrieval(parsed_listing_url):
    string_data = parsed_listing_url.find_all(class_='row_info')[1].text.strip()
    road_tax_yearly = road_tax_error_handler(string_data)
    
    return road_tax_yearly
    

# Registered Date Retriever
def registered_date_retrieval(parsed_listing_url):
    reg_date = parsed_listing_url.find_all(class_='row_bg')[1].find_all('td')[3].text.split()[0].split('(')[0]
    return reg_date

# Days of COE Retriever
def days_of_coe_retrieval(parsed_listing_url):
    days_of_coe_left_yy_mm_dd_format_for_cleaner_function=\
    parsed_listing_url.find_all(class_='row_bg')[1].find_all('td')[3].text.split('(')[1].split('COE')[0].strip()
    
    return yr_mm_dd_cleaner(days_of_coe_left_yy_mm_dd_format_for_cleaner_function)


# Define a function to calculate days of COE left
def yr_mm_dd_cleaner(str1):
    """Accepts a string that may or may include the elements yr mths days and 
    converts the whole string into number of days.
    ----
    Input: single string
    output: number of days in integer form
    ----
    Example string inputs:
    - 4yrs 2mths 23days
    - 5yrs
    - 2 mths 23 days
    - 50 days
    """
    
    # Convert days_of_coe_left_yy_mm_dd to days    
    year_index = str1.find('yr')
    if year_index == -1:
        year = 0
    else:
        year = int(str1[year_index-1])

        
    mth_index = str1.find('mth')
    if mth_index == -1:
        mth = 0
    else:
        mth = int(str1[mth_index-1])

        
    day_index = str1.find('day')
    if day_index == -1:
        day = 0
    else:
        day = int(str1[day_index-1])
       
    days_of_coe_left = (year * 365) + (mth * 30) + day 
    return days_of_coe_left


# Mileage Retriever
def mileage_error_handler(data_value):
    if len(data_value) < 2:  # Deals with ['na'] scenarios
        mileage_km = np.nan  # Stores NA values as nan

    else:  
        try:                 
            mileage_km = int(data_value[0].strip().split(',')[0] + data_value[0].strip().split(',')[1])
        except IndexError: # Will fail on IndexError if tries to split '900' with a ',' in ['',900]
            mileage_km = int(data_value[0].strip())
    
    return mileage_km

def mileage_retrieval(parsed_listing_url):
        
    data_value = parsed_listing_url.find_all(class_='row_info')[0].text.strip()
    data_value = data_value.split('km')
    mileage_km = mileage_error_handler(data_value)
    
    return mileage_km

# Manufactured Year Retriever
def manufactured_year_retrieval(parsed_listing_url):
    manufactured_year = parsed_listing_url.find_all(class_='row_info')[6].text.strip()
    return manufactured_year.split()[0]

# Transmission Retriever
def transmission_retrieval(parsed_listing_url):
    transmission = parsed_listing_url.find_all(class_='row_info')[7].text.strip()
    return transmission.split()[0]

# Deregistration Value Retriever
def dereg_value_retrieval(parsed_listing_url):
    # Splits into ['NA'], or ['$11,026', 'as', 'of', 'today', '(change)'] or ['$900', 'as', 'of', 'today', '(change)']
    data_value = parsed_listing_url.find_all(class_='row_info')[2].text.strip().split() 
    
    dereg_value_from_scrape_date = dereg_value_error_handler(data_value)
    return dereg_value_from_scrape_date
    

def dereg_value_error_handler(data_value):
    if len(data_value) < 2:  # Deals with ['NA'] scenario
        dereg_value_from_scrape_date = np.nan

    else: 
        data_value = data_value[0].split('$')[1] # Puts input into '11,026' or '900' format
        try:                 
            dereg_value_from_scrape_date = \
            int(data_value.split(',')[0] +\
                data_value.split(',')[1]) # Will fail on IndexError if tries to split '900' with a ',' in ['',900]
        except IndexError: 
            dereg_value_from_scrape_date = int(data_value.strip())

        return dereg_value_from_scrape_date


# Open Market Value Retriever
def omv_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        omv = np.nan

    else:
        try:
            omv = int(data_value[1].split(',')[0] +\
                      data_value[1].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            omv = int(data_value[1])
    return omv


def omv_retrieval(parsed_listing_url):    
    data_value = parsed_listing_url.find_all(class_='row_info')[8].text.split('$') 
    # Splits data into ['', '21,967'], ['','900'] or ['NA'] format for input into error function
    
    omv = omv_error_handler(data_value)
    return omv     

# ARF Retriever
def error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        desired_value = np.nan

    else:
        try:
            desired_value = int(data_value[1].split(',')[0] +\
                                data_value[1].split(',')[1])   # Will fail on index error if try to split 900
        except IndexError:
            desired_value = int(data_value[1])
    return desired_value


def arf_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[9].text.split('$')
    arf = error_handler(data_value)
    return arf

# COE Price retriever 
def coe_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        coe_from_scrape_date = np.nan

    else:
        try:
            coe_from_scrape_date = int(data_value[1].split(',')[0] +\
                                       data_value[1].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            coe_from_scrape_date = int(data_value[1])
    return coe_from_scrape_date


def coe_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[3].text.split('$')
    coe_from_scrape_date = coe_error_handler(data_value)
    return coe_from_scrape_date

# Engine Capacity Retriever
def engine_capacity_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        desired_value = np.nan

    else:
        try:
            desired_value = int(data_value[0].split(',')[0] +\
                                       data_value[0].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            desired_value = int(data_value[0])
    return desired_value


def engine_capacity_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[4].text.strip().split('cc')
    engine_capacity = engine_capacity_error_handler(data_value)
    return engine_capacity

# Curb Weight Retriever
def curb_weight_error_handler(data_value):
    if len(data_value) < 2:  # deals iwth ['NA'] input
        desired_value = np.nan

    else:
        try:
            desired_value = int(data_value[0].split(',')[0] +\
                                       data_value[0].split(',')[1])  # Will fail on index error if try to split 900
        except IndexError:
            desired_value = int(data_value[0])
    return desired_value


def curb_weight_retrieval(parsed_listing_url):
    data_value = parsed_listing_url.find_all(class_='row_info')[5].text.split()
    curb_weight = curb_weight_error_handler(data_value)
    return curb_weight

# Number of owners retriever
def number_of_owners_retrieval(parsed_listing_url):
    no_of_owners = int(parsed_listing_url.find_all(class_='row_info')[-1].text)
    return no_of_owners


# Type of Vehicle Retriever
def type_of_vehicle_retrieval(parsed_listing_url):
    type_of_vehicle = parsed_listing_url.find(class_='row_bg1').find_all('a')[0].text 
    return type_of_vehicle

## 4. Get Links For All Postings

Links for all the car postings will be stored in a list before accessing them one by one for data extraction

In [3]:
# Create listings URLs to iterate through
main_page_listing_list = [] # create list to store search pages
for idx, link in enumerate(range(100)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(idx * 100) + "&RPG=100&AVL=2&VEH=2" #search by of 100 car listings per page
    main_page_listing_list.append(url)

In [4]:
print(main_page_listing_list,'\n','\n', len(main_page_listing_list))

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=200&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=300&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=400&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=500&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=600&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=700&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=800&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=900&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1000&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/li

## 5. Retrieval of Individual Listing URLs from Search Pages

In [ ]:
# Base url, or you can think of this as the individual car listing prefix
base_url = 'https://www.sgcarmart.com/used_cars/'
listing_urls = []

# Acquiring indvidual car listings    
for main_link in main_page_listing_list:
   
    # Make a request to the website and get the object
    content = requests.get(main_link)

    # Parse the HTML text
    soup = BeautifulSoup(content.text, 'lxml')

    # Find every single URL in the webpage , refer to this post: # https://stackoverflow.com/questions/46490626/getting-all-links-from-a-page-beautiful-soup
    # This returns a list of every tag that contains a link in one main link (each element in main page listing)
    links = soup.find_all('a')
    
    # Create a list for storing all the individual listing urls
    
    for link in links:
        # Get link in <a href>
        suffix = link.get('href')

        # Check if 'ID=' and 'DL=' exist in the string
        if ('ID=' in suffix) and ('DL=' in suffix):

            # Concatenate the two strings if they do
            listing_url = base_url + suffix
        #    print(listing_url)
            
            # Append result to the list
            listing_urls.append(listing_url)
            
#     Removing duplicates
    set_listing_urls = set(listing_urls)
    listing_urls = list(set_listing_urls)
    
    # Prevent oneself from getting blocked from the website
    time.sleep(3)

In [ ]:
print(len(listing_urls))
print(len(set(listing_urls)))
print(len(list(set(listing_urls))))

In [ ]:
print(listing_urls[:10])

## 6. Create DataFrame

In [ ]:
# Creating an empty DataFrame for attributes of interest
df = pd.DataFrame(columns=['LISTING_URL', 'BRAND', 'PRICE', 'DEPRE_VALUE_PER_YEAR',
       'REG_DATE', 'MILEAGE_KM', 'MANUFACTURED_YEAR',
       'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       'SCRAPE_DATE', 'OMV', 'ARF', 'COE_FROM_SCRAPE_DATE',
       'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 'CURB_WEIGHT_KG',
       'NO_OF_OWNERS', 'VEHICLE_TYPE'])

In [ ]:
filename = 'sgcarmart_used_cars_prices'
i = 0 # Indexing rows in the DF

for listingurl in listing_urls:
    response = requests.get(listingurl)
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    print(listing_url)

    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listingurl
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)
    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan
    
    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan

    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
        
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan

        
    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan

    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan
        
    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
        
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
        
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
        
    df.to_csv("{}.csv".format(filename))    
        
    i += 1 # Allows next car listing to be put into a next row in the dataframe
    time.sleep(5)  # Prevents us from getting locked out of the website
    

In [ ]:
df = pd.read_csv('sgcarmart_used_cars_prices.csv',index_col=0)
df